### Add Fumehood Data
This notebook queries raw data from WebCTRL, calculates fumehood energy and sash position data, and adds this data into Steve's SQL database.

In [1]:
import random
import time
import requests
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil import tz
import matplotlib.pyplot as plt

### Load Point Names
First, we load the point names of each building, as these will be used to make query requests for each lab and to construct the layout of buildings, floors, labs, etc

In [2]:
biotech = pd.read_csv("./Point Names/biotech point names.csv")
biotech

,building,hood,lab,floor,server,flow_sensor,sash_position_sensor,hood_occ_sensor,room_occ_sensor,internal_temp_sensor,external_temp_sensor
0,biotech,1,G54,G,biotech_main,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,NaN,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/ground_flr_mech/building_hydronic_hea...
1,biotech,1,141,1,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,NaN,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...
2,biotech,1,143,1,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,NaN,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...
3,biotech,1,147B,1,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,NaN,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...
4,biotech,1,153,1,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,NaN,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...
...,...,...,...,...,...,...,...,...,...,...,...
69,biotech,1,445B,4,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,NaN,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/ground_flr_mech/building_hydronic_hea...
70,biotech,1,453,4,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,NaN,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/ground_flr_mech/building_hydronic_hea...
71,biotech,1,453B,4,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,NaN,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/ground_flr_mech/building_hydronic_hea...
72,biotech,1,457,4,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,NaN,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/ground_flr_mech/building_hydronic_hea...


### Query Function

In [4]:
def fume_query(target, server, start, end, aggType):
    # targets_req = []

    # for target in targets:
    #     data = {}
    #     data["payload"] = {}
    #     data["payload"]["schema"] = server
    #     # data["payload"]["additional"] = [aggType]
    #     data["target"] = target
    #     targets_req.append(data)

    url = "https://ypsu0n34jc.execute-api.us-east-1.amazonaws.com/dev/query"
    data = {
        "range": {
            "from": start,
            "to": end,
        },
        "targets": [
        {
          "payload": {
            "schema": server,
            "additional": [
                    aggType,
                ]
          },
          "target": target
        }
      ],
    }
    response = requests.post(url, json=data)
    print(response)
    print(response.json())
    print(len(response.json()))

    master = pd.json_normalize(response.json(), record_path="datapoints", meta=["target"]).rename(columns={0: "value", 1: "timestamp"}).set_index("target").rename_axis(None)
    # Remove the rows where the metric is None (i.e., do not show the averaged rows because this is not useful)
    # master = master[~master["metric"].isna()]
    master["timestamp"] = master["timestamp"].astype("datetime64[ms]").map(lambda x: x.to_pydatetime().replace(tzinfo=tz.tzutc()).astimezone(tz.tzlocal()))

    return master

### Metrics Functions
These functions calculate the metrics we want (time sash open and energy usage) from the raw WebCTRL data

In [5]:
def total_time_sash_open(sash_points, server, start, end, aggType):
  df = fume_query(sash_points, server, start, end, aggType)

  # display(df)

  time_interval = df["timestamp"][1].minute - df["timestamp"][0].minute
  # print("Time interval", time_interval)

  # display(df)
  # from running the above on a large time difference, 1.2 inches is the most common smallest value
  df["time_open_mins"] = np.where((df["value"] > 1.2), time_interval, 0)
  # display(df)

  df = df.dropna()
  # display(df)

  df = df.groupby(pd.Grouper(key="timestamp", freq='15Min', label='right')).sum()
  df.index = df.index.map(lambda x: x.to_pydatetime().timestamp())
  display(df)

  return df["time_open_mins"]

In [6]:
# Arguments: CFM Point, Sash Point, Occ Point, Internal Temp Point, External Temp Point, Server Name, Start Time, End Time
# Returns: Total energy from hume hood, aggregated by hour.

# A switch in the energy calculations that allows us to avoid issues between times of years where it is hotter/colder outside.
def coldorhot(cfm, external, internal, time_interval):
    if external<=internal:
        #sensible heating equation
        return round(1.08 * cfm * (internal - external) / (60 / time_interval), 3)
    if external>internal:
        #enthalpy of air
        return round(0.24 * cfm /13.333 * 60 * (external - internal) / (60 / time_interval), 3)

def total_energy(cfm_point, sash_point, occ_point, internal_temp_point, external_temp_point, server, start, end):
  #external_temp_master = outside_temp(start,end)
  cfm_list = query_to_list(cfm_point, server, start, end)
  sash_list = query_to_list(sash_point, server, start, end)
  occ_list = query_to_list(occ_point, server, start, end)
  internal_temp_list = query_to_list(internal_temp_point, server, start, end)
  external_temp_list = query_to_list(external_temp_point, server, start, end)

  df = pd.concat([cfm_list, sash_list, occ_list, internal_temp_list, external_temp_list], axis=1)
  df.columns = ["cfm", "sash", "occ", "internal_temp", "external_temp"]
  display(df)

  df['minute'] = df.index.map(lambda x: x.to_pydatetime().minute)
  df = df[df['minute'].isin([0, 15, 30, 45])]
  display(df)

  df["external_temp"] = df["external_temp"].interpolate()
  display(df)
  
  time_interval = df.index[1].minute - df.index[0].minute

  df['BTU'] = df.apply(lambda df: coldorhot(df['cfm'], df['external_temp'], df['internal_temp'], time_interval=time_interval), axis=1)
  
  df.index = df.index.map(lambda x: x.to_pydatetime().replace(tzinfo=tz.tzutc()).astimezone(tz.tzlocal()))
  display(df)

  # df = df.groupby(pd.Grouper(freq='60Min', label='right')).sum()
#  df.index = df.index.strftime('%s').astype("int")
  df.index = df.index.map(lambda x: x.to_pydatetime().timestamp())
  display(df)

  unocc_df = df[df['occ']==0]
  occ_df = df[df['occ']==1]

  return (unocc_df["sash"], occ_df["sash"], unocc_df["BTU"], occ_df["BTU"])

### Insert into Database Function
This function takes in the name of a "pseudo point" and inserts the relevant data into Steve's SQL database.

In [14]:
# the x-api-key is the secret authentication to allow posting updates
headers = {"x-api-key": "LfSpEZ3woOuljzb2EAg0"}
# the endpoint URL for trend updates
TREND_API_URL = "https://7zhs2wplt9.execute-api.us-east-1.amazonaws.com/dev/trend"
# time in seconds that the request will timeout if no response is received.
# the API will only run for 6 seconds, so if it can't process
# the data posted in that amount of time it will fail.
API_TIMEOUT = 30

# timestamps
now = int(time.time())
last_hour = now - now % 3600
start_time = last_hour - (3600 * 6)

def insert_into_database(pseudo_point, data):
    # data = [compute_value(point, ts) for ts in range(start_time, last_hour, 3600)]
    data = np.fliplr(data.reset_index().to_numpy()).tolist()
    print(data)

    json_obj = {"object_name": pseudo_point, "data": data }
    # e.g.
    # { "object_name": "Biotech.GroundFloor.Lab_g54.Hood_1.energy.occ",
    # "data": [[0, 1681462800], [0, 1681466400], [0, 1681470000], [4.430475, 1681473600]],
    # }
    req_start = time.time()
    response = requests.post(TREND_API_URL, json=json_obj, headers=headers, timeout=API_TIMEOUT)
    req_end = time.time()
    r = response.json()
    print(f"""received status_code:{response.status_code} msg:{r.get('message', None)}
    rows:{r.get('affectedRows', None)} duplicates:{r.get('duplicates', None)} 
    for point:{pseudo_point} with id:{r.get('obj_id', None)} in {req_end-req_start:.3f} seconds.""")

### Insert into Database
This cell actually runs the code to insert the data processed above into Steve's SQL database

TODO
1. Load the namingconvention.csv to obtain the structure of biotech along with all points
2. For each row (fume hood) of the csv, pass the point names into the queries.  For example, pass the point names for the current hood to calculate a total_time_sash_open for that hood (do this for occ and unocc).  Do this for both the sash open and energy metrics.
3. After doing this, also for each row, insert the data into Steve's database using the correct pseudo-point name (the last four rows of the csv)

- Start this at the beginning of 2023


In [19]:
print("DOWNLOADING DATA")
# trend_data = total_energy(cfm_point="#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_441_control/hoodvalve_flow/trend_log",
#                                  sash_point="#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_441_control/hood_sash",
#                                  occ_point="#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_441_control/occ",
#                                  internal_temp_point="#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_441_control/zone/zone_temp/trend_log",
#                                  external_temp_point="#biotech/ground_flr_mech/building_hydronic_heating_syatems/reheat_heat_exchanger/oat",
#                                  server="biotech_main",
#                                  start=str(datetime(2024, 3, 15)),
#                                  end=str(datetime.now()))

# for idx, row in biotech.iterrows():
#     sashOpenData = total_time_sash_open(sash_points=row["sash_position_sensor"],
#                                     server=row["server"], start=str(datetime(2024, 9, 15)),
#                                  end=str(datetime.now()), aggType="noagg")
    
#     print("INSERTING SASH TIME INTO DATABASE")
#     insert_into_database(str(row['building']).capitalize() + \
#                             ".Floor_" + str(row['floor']) + \
#                             ".Lab_" + str(row['lab']).lower() + \
#                             ".Hood_" + str(row['hood']) + ".sashOpenTime.unocc", sashOpenData)

insert_into_database("Biotech.Floor_1.Lab_153b.Hood_1.sashOpenTime.unocc", total_time_sash_open(sash_points="#biotech/biotech_1st_floor/first_floor_fume_hood_lab_spaces/lab_153b_control/hood_sash",
                                    server="biotech_main", start=str(datetime(2024, 7, 15)),
                                 end=str(datetime.now()), aggType="noagg"))

# print("INSERTING SASH OCC INTO DATABASE")
# insert_into_database("Biotech.Floor_4.Lab_441.Hood_1.sashOpenTime.occ", trend_data[1])

# print("INSERTING ENERGY UNOCC INTO DATABASE")
# insert_into_database("Biotech.Floor_4.Lab_441.Hood_1.energy.unocc", trend_data[2])

# print("INSERTING ENERGY OCC INTO DATABASE")
# insert_into_database("Biotech.Floor_4.Lab_441.Hood_1.energy.occ", trend_data[3])

# a list of points
points = [
    # "Biotech.Floor_4.Lab_433.Hood_1.energy.occ",
    # "Biotech.Floor_4.Lab_433.Hood_1.energy.unocc",
    # "Biotech.Floor_4.Lab_433.Hood_1.sashOpenTime.occ",
    # "Biotech.Floor_4.Lab_433.Hood_1.sashOpenTime.unocc",
    # "Biotech.GroundFloor.Lab_g54.Hood_1.energy.unocc",
    # "Biotech.GroundFloor.Lab_g54.Hood_1.sashOpenTime.occ",
]

DOWNLOADING DATA
<Response [200]>
[{'target': 'ABSPATH:1:#biotech/biotech_1st_floor/first_floor_fume_hood_lab_spaces/lab_153b_control/hood_sash', 'datapoints': [[6.2, 1721016000000], [6.2, 1721016900000], [6.2, 1721017800000], [6.2, 1721018700000], [6.2, 1721019600000], [6.2, 1721020500000], [6.2, 1721021400000], [6.2, 1721022300000], [6.2, 1721023200000], [6.2, 1721024100000], [6.2, 1721025000000], [6.2, 1721025900000], [6.2, 1721026800000], [6.2, 1721027700000], [6.2, 1721028600000], [6.2, 1721029500000], [6.2, 1721030400000], [6.2, 1721031300000], [6.2, 1721032200000], [6.2, 1721033100000], [6.2, 1721034000000], [6.2, 1721034900000], [6.2, 1721035800000], [6.2, 1721036700000], [6.2, 1721037600000], [6.2, 1721038500000], [6.2, 1721039400000], [6.2, 1721040300000], [6.2, 1721041200000], [6.2, 1721042100000], [6.2, 1721043000000], [6.2, 1721043900000], [6.2, 1721044800000], [6.2, 1721045700000], [6.2, 1721046600000], [6.2, 1721047500000], [6.2, 1721048400000], [6.2, 1721049300000], [6.

/var/folders/h7/th7jww813155ckgv05q_ckzr0000gn/T/ipykernel_2090/433842460.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  time_interval = df["timestamp"][1].minute - df["timestamp"][0].minute


,value,time_open_mins
timestamp,,
1.721017e+09,6.2,15
1.721018e+09,6.2,15
1.721019e+09,6.2,15
1.721020e+09,6.2,15
1.721020e+09,6.2,15
...,...,...
1.726744e+09,1.7,15
1.726744e+09,1.7,15
1.726745e+09,1.7,15


[[15.0, 1721016900.0], [15.0, 1721017800.0], [15.0, 1721018700.0], [15.0, 1721019600.0], [15.0, 1721020500.0], [15.0, 1721021400.0], [15.0, 1721022300.0], [15.0, 1721023200.0], [15.0, 1721024100.0], [15.0, 1721025000.0], [15.0, 1721025900.0], [15.0, 1721026800.0], [15.0, 1721027700.0], [15.0, 1721028600.0], [15.0, 1721029500.0], [15.0, 1721030400.0], [15.0, 1721031300.0], [15.0, 1721032200.0], [15.0, 1721033100.0], [15.0, 1721034000.0], [15.0, 1721034900.0], [15.0, 1721035800.0], [15.0, 1721036700.0], [15.0, 1721037600.0], [15.0, 1721038500.0], [15.0, 1721039400.0], [15.0, 1721040300.0], [15.0, 1721041200.0], [15.0, 1721042100.0], [15.0, 1721043000.0], [15.0, 1721043900.0], [15.0, 1721044800.0], [15.0, 1721045700.0], [15.0, 1721046600.0], [15.0, 1721047500.0], [15.0, 1721048400.0], [15.0, 1721049300.0], [15.0, 1721050200.0], [15.0, 1721051100.0], [15.0, 1721052000.0], [15.0, 1721052900.0], [15.0, 1721053800.0], [15.0, 1721054700.0], [15.0, 1721055600.0], [15.0, 1721056500.0], [15.0, 17

### Testing
These cells are used for testing purposes

In [48]:
def synthetic_query(target, server, start, end):
    url = "https://portal.emcs.cornell.edu/api/datasources/proxy/5/query"
    data = {
      "range": {
        "from": start,
        "to": end,
      },
      "targets": [
        {
          "payload": {
            "schema": server,
            "additional": [
                    "aggD"
                ]
          },
          "target": target
        },
        {
          "payload": {
            "schema": server,
            "additional": [
                    "aggD",
                ]
          },
          "target": "Biotech.Floor_4.Lab_403.Hood_1.sashOpenTime.unocc"
        },
        {
          "payload": {
            "schema": server,
            "additional": [
                    "aggD",
                ]
          },
          "target": "Biotech.Floor_4.Lab_403b.Hood_1.sashOpenTime.unocc"
        },
        {
          "payload": {
            "schema": server,
            "additional": [
                    "aggD",
                ]
          },
          "target": "Biotech.Floor_4.Lab_409.Hood_1.sashOpenTime.unocc"
        },
        {
          "payload": {
            "schema": server,
            "additional": [
                    "aggD",
                ]
          },
          "target": "Biotech.Floor_4.Lab_409b.Hood_1.sashOpenTime.unocc"
        },
        {
          "payload": {
            "schema": server,
            "additional": [
                    "aggD",
                ]
          },
          "target": "Biotech.Floor_4.Lab_445b.Hood_1.sashOpenTime.unocc"
        },
        {
          "payload": {
            "schema": server,
            "additional": [
                    "aggD",
                ]
          },
          "target": "Biotech.Floor_4.Lab_417.Hood_1.sashOpenTime.unocc"
        },
        {
          "payload": {
            "schema": server,
            "additional": [
                    "aggD",
                ]
          },
          "target": "Biotech.Floor_4.Lab_418.Hood_1.sashOpenTime.unocc"
        },
        {
          "payload": {
            "schema": server,
            "additional": [
                    "aggD",
                ]
          },
          "target": "Biotech.Floor_4.Lab_421.Hood_1.sashOpenTime.unocc"
        },
        {
          "payload": {
            "schema": server,
            "additional": [
                    "aggD",
                ]
          },
          "target": "Biotech.Floor_4.Lab_421b.Hood_1.sashOpenTime.unocc"
        },
        {
          "payload": {
            "schema": server,
            "additional": [
                    "aggD",
                ]
          },
          "target": "Biotech.Floor_4.Lab_427b.Hood_1.sashOpenTime.unocc"
        },
        {
          "payload": {
            "schema": server,
            "additional": [
                    "aggD",
                ]
          },
          "target": "Biotech.Floor_4.Lab_433.Hood_1.sashOpenTime.unocc"
        },
        {
          "payload": {
            "schema": server,
            "additional": [
                    "aggD", "sum"
                ]
          },
          "target": "Biotech.Floor_1.Lab_141.Hood_1.sashOpenTime.unocc"
        },
      ],
    }
    response = requests.post(url, json=data)
    print(response)
    print(response.json())
    print(len(response.json()))

    master = pd.json_normalize(response.json(), record_path="datapoints", meta=["target", "metric"]).rename(columns={0: "value", 1: "timestamp"}).set_index("target").rename_axis(None)
    # Remove the rows where the metric is None (i.e., do not show the averaged rows because this is not useful)
    master = master[~master["metric"].isna()]
    master["timestamp"] = master["timestamp"].astype("datetime64[ms]").map(lambda x: x.to_pydatetime().replace(tzinfo=tz.tzutc()).astimezone(tz.tzlocal()))

    master[["building", "floor", "lab", "hood"]] = [i[0:4] for i in master.index.str.split(".")]
    master[["floor", "lab", "hood"]] = master[["floor", "lab", "hood"]].replace(r'^.*?_', '', regex=True)
    
    # display(master)

    return master

sash_open_time = synthetic_query(target="Biotech.Floor_2.Lab_203b.Hood_1.sashOpenTime.unocc", server="biotech_main",
                                    start=str(datetime(2024, 5, 15)),
                                    end=str(datetime.now()))

# final_df = sash_open_time.fillna(0)
display(sash_open_time)
# plt.scatter(sash_open_time["timestamp"], sash_open_time["value"])

<Response [200]>
[{'target': 'Biotech.Floor_1.Lab_141.Hood_1.sashOpenTime.unocc.sum', 'metric': 'sum', 'datapoints': [[1185, 1715731200000], [1440, 1715817600000], [1440, 1715904000000], [1440, 1715990400000], [1425, 1716076800000], [1440, 1716163200000], [1440, 1716249600000], [1440, 1716336000000], [1440, 1716422400000], [1440, 1716508800000], [1440, 1716595200000], [1440, 1716681600000], [1440, 1716768000000], [1440, 1716854400000], [1440, 1716940800000], [1440, 1717027200000], [1440, 1717113600000], [1440, 1717200000000], [1440, 1717286400000], [1440, 1717372800000], [1440, 1717459200000], [1440, 1717545600000], [1440, 1717632000000], [1440, 1717718400000], [330, 1717804800000]]}, {'target': 'Biotech.Floor_1.Lab_141.Hood_1.sashOpenTime.unocc', 'datapoints': [[15, 1715731200000], [15, 1715817600000], [15, 1715904000000], [15, 1715990400000], [14.84375, 1716076800000], [15, 1716163200000], [15, 1716249600000], [15, 1716336000000], [15, 1716422400000], [15, 1716508800000], [15, 171659

,value,timestamp,metric,building,floor,lab,hood
Biotech.Floor_1.Lab_141.Hood_1.sashOpenTime.unocc.sum,1185.0,2024-05-14 20:00:00-04:00,sum,Biotech,1,141,1
Biotech.Floor_1.Lab_141.Hood_1.sashOpenTime.unocc.sum,1440.0,2024-05-15 20:00:00-04:00,sum,Biotech,1,141,1
Biotech.Floor_1.Lab_141.Hood_1.sashOpenTime.unocc.sum,1440.0,2024-05-16 20:00:00-04:00,sum,Biotech,1,141,1
Biotech.Floor_1.Lab_141.Hood_1.sashOpenTime.unocc.sum,1440.0,2024-05-17 20:00:00-04:00,sum,Biotech,1,141,1
Biotech.Floor_1.Lab_141.Hood_1.sashOpenTime.unocc.sum,1425.0,2024-05-18 20:00:00-04:00,sum,Biotech,1,141,1
...,...,...,...,...,...,...,...
Biotech.Floor_4.Lab_445b.Hood_1.sashOpenTime.unocc.sum,0.0,2024-06-02 20:00:00-04:00,sum,Biotech,4,445b,1
Biotech.Floor_4.Lab_445b.Hood_1.sashOpenTime.unocc.sum,0.0,2024-06-03 20:00:00-04:00,sum,Biotech,4,445b,1
Biotech.Floor_4.Lab_445b.Hood_1.sashOpenTime.unocc.sum,0.0,2024-06-04 20:00:00-04:00,sum,Biotech,4,445b,1
Biotech.Floor_4.Lab_445b.Hood_1.sashOpenTime.unocc.sum,0.0,2024-06-05 20:00:00-04:00,sum,Biotech,4,445b,1


In [48]:
sash_open_time.groupby(sash_open_time.index).sum()

/var/folders/3h/x3_4y46j1k1g_yxjmjq3ktxh0000gn/T/ipykernel_13500/1628084180.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sash_open_time.groupby(sash_open_time.index).sum()


,value
Biotech.Floor_1.Lab_141.Hood_1.sashOpenTime.unocc.sum,19905
Biotech.Floor_4.Lab_403.Hood_1.sashOpenTime.unocc.sum,19905
Biotech.Floor_4.Lab_403b.Hood_1.sashOpenTime.unocc.sum,19905
Biotech.Floor_4.Lab_409.Hood_1.sashOpenTime.unocc.sum,19905
Biotech.Floor_4.Lab_409b.Hood_1.sashOpenTime.unocc.sum,19905
Biotech.Floor_4.Lab_417.Hood_1.sashOpenTime.unocc.sum,19905
Biotech.Floor_4.Lab_418.Hood_1.sashOpenTime.unocc.sum,19905
Biotech.Floor_4.Lab_421.Hood_1.sashOpenTime.unocc.sum,19905
Biotech.Floor_4.Lab_421b.Hood_1.sashOpenTime.unocc.sum,19905
Biotech.Floor_4.Lab_427b.Hood_1.sashOpenTime.unocc.sum,19905


In [34]:
# Calculate total time sash open for DynamoDB database
# Remove time between 7 PM - 7 AM
last_day = final_df.groupby(pd.Grouper(freq='1D', label='right')).sum()
display(last_day)

,Sash Open Time
2024-05-16 00:00:00-04:00,360.0
2024-05-17 00:00:00-04:00,360.0
2024-05-18 00:00:00-04:00,360.0
2024-05-19 00:00:00-04:00,360.0
2024-05-20 00:00:00-04:00,360.0
2024-05-21 00:00:00-04:00,360.0
2024-05-22 00:00:00-04:00,360.0
2024-05-23 00:00:00-04:00,360.0
2024-05-24 00:00:00-04:00,360.0
2024-05-25 00:00:00-04:00,360.0
